In [ ]:
import os, re
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoTokenizer

# Drive 마운트
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    print("Google Drive 마운트 완료.")
except ImportError:
    print("Google Colab 환경이 아닙니다. 로컬 경로를 직접 설정하세요.")

Mounted at /content/drive
Google Drive 마운트 완료.


In [ ]:
# ====================
# 1. Text Cleaning
# ====================
def clean_text(text):
    """공백/개행 제거 + 문자열 정규화"""
    if pd.isna(text):
        return ""
    return re.sub(r"\s+", " ", str(text)).strip()

In [ ]:
# ====================
# 2. 문장 병합
# ====================
def merge_sentences(df):
    """사람문장1~3을 안전하게 병합"""
    sentence_cols = [c for c in ["사람문장1", "사람문장2", "사람문장3"] if c in df.columns]

    df[sentence_cols] = df[sentence_cols].fillna("").astype(str)

    df["merged_text"] = df[sentence_cols].agg(" ".join, axis=1).apply(clean_text)

    # 완전히 빈 텍스트 제거
    df = df[df["merged_text"] != ""]
    return df

In [ ]:
# ==============================
# 3. Multi-task Label Encoder
# ==============================
def encode_labels(train_df, valid_df, save_path):
    """감정 + 상황 인코딩"""
    # Emotion
    emo_labels = sorted(train_df["감정_대분류"].unique())
    emo2id = {label: idx for idx, label in enumerate(emo_labels)}

    # Situation
    sit_labels = sorted(train_df["상황키워드"].unique())
    sit2id = {label: idx for idx, label in enumerate(sit_labels)}

    # Apply
    for df in (train_df, valid_df):
        df["label_emotion"] = df["감정_대분류"].map(emo2id)
        df["label_situation"] = df["상황키워드"].map(sit2id)

    # 저장
    label_map = {
        "emotion2id": emo2id,
        "situation2id": sit2id,
        "id2emotion": {v: k for k, v in emo2id.items()},
        "id2situation": {v: k for k, v in sit2id.items()}
    }
    with open(os.path.join(save_path, "label_map.pkl"), "wb") as f:
        pickle.dump(label_map, f)

    print(f"[INFO] Emotion Classes: {len(emo_labels)}, Situation Classes: {len(sit_labels)}")

    return train_df, valid_df

In [ ]:
# ===============
# 5. Save CSV
# ===============
def save_csv(train_df, valid_df, save_path):
    os.makedirs(save_path, exist_ok=True)

    train_df.to_csv(os.path.join(save_path, "train.csv"), index=False, encoding="utf-8-sig")
    valid_df.to_csv(os.path.join(save_path, "valid.csv"), index=False, encoding="utf-8-sig")

    print(f"[INFO] CSV Saved → {save_path}")

In [ ]:
# =====================
# 6. Main Pipeline
# =====================
def run_preprocessing_multitask(
    train_path,
    valid_path,
    save_path
):
    print("\n===== Multi-Task Preprocessing Start =====")

    # Load & Merge
    train_df = merge_sentences(pd.read_excel(train_path))
    valid_df = merge_sentences(pd.read_excel(valid_path))

    # Encode Labels
    train_df, valid_df = encode_labels(train_df, valid_df, save_path)

    # Save CSV
    save_csv(train_df, valid_df, save_path)

    print("===== Preprocessing Completed =====\n")
    return train_df, valid_df

In [ ]:
PROJ_PATH = "/content/drive/MyDrive/Projects/MindLog"
SAVE_PATH = os.path.join(PROJ_PATH, "processed")
DATA_PATH = os.path.join(PROJ_PATH, "data")

run_preprocessing_multitask(
    train_path=os.path.join(DATA_PATH, "감성대화말뭉치_Training.xlsx"),
    valid_path=os.path.join(DATA_PATH, "감성대화말뭉치_Validation.xlsx"),
    save_path=SAVE_PATH
)


===== Multi-Task Preprocessing Start =====
[INFO] Emotion Classes: 6, Situation Classes: 12
[INFO] CSV Saved → /content/drive/MyDrive/Projects/MindLog/processed
===== Preprocessing Completed =====



(       Unnamed: 0  연령  성별     상황키워드    신체질환 감정_대분류 감정_소분류  \
 0               1  청년  여성  진로,취업,직장    해당없음     분노  노여워하는   
 1               2  청년  여성  진로,취업,직장    해당없음     분노  노여워하는   
 2               3  청년  여성  진로,취업,직장    해당없음     분노  노여워하는   
 3               4  청년  여성  진로,취업,직장    해당없음     분노  노여워하는   
 4               5  청년  여성  진로,취업,직장    해당없음     분노  노여워하는   
 ...           ...  ..  ..       ...     ...    ...    ...   
 51625       51626  노년  남성        재정  만성질환 무     분노    성가신   
 51626       51627  노년  여성        재정  만성질환 무     불안    초조한   
 51627       51628  노년  여성        재정  만성질환 무     상처    희생된   
 51628       51629  노년  여성      대인관계  만성질환 무     불안  걱정스러운   
 51629       51630  노년  여성      대인관계  만성질환 무     상처   배신당한   
 
                                                    사람문장1  \
 0                              일은 왜 해도 해도 끝이 없을까? 화가 난다.   
 1         이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.   
 2      회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...   
 3      직장